In [1]:
import requests
from bs4 import BeautifulSoup
import re
import csv
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

## 주식 종목

In [84]:
code = {"삼성전자": ["005930",1],
        "LG": ["003550",1],
        "SK하이닉스": ["000660",1],
        "LG전자": ["066570",1],
        "삼성전기": ["009150",1],
        "LG이노텍": ["011070",1],
        "삼성SDI": ["006400",2],
        "LG디스플레이": ["034220",2],
        "SK머티리얼즈": ["036490",3],
        "SK이노베이션": ["096770",4],
        "LG화학": ["051910",4],
        "에코프로비엠": ["247540",4],
        "Naver": ["035420", 5],
        "카카오": ["035720", 5],
        "이마트": ["139480",6],
        "신세계": ["004170", 6],
        "롯데푸드": ["002270", 7],
        "신세계푸드": ["031440", 7],
        "CJ제일제당": ["097950",7],
        "효성": ["004800",8],
        "효성티앤씨": ["298020",8],
        "효성첨단소재" : ["298050",8],
        "효성화학": ["298000",8],
        "셀트리온":["068270",9],
        "셀트리온헬스케어":["091990",9],
        "셀트리온제약":["068760",9],
        "셀리버리":["268600",9],
        "SK바이오사이언스":["302440",9],
        "SK바이오팜":["326030",9],
        "삼성바이오로직스":["207940",9],
        "현대차":["005380",10],
        "현대모비스":["012330",10],
        "기아":["000270",10],
        "POSCO":["005490",11],
        "포스코강판":["058430",11],
        "포스코인터내셔널":["047050",11,"A"],
        "포스코케미칼":["003670",11],
        "포스코ICT":["022100",11],
        "삼성물산":["028260",1,"A"],
        "LX인터내서널":["001120",1,"A"],
        }

## url

In [85]:
urls = []
for i in code.values():
    urls.append("https://finance.naver.com/item/main.nhn?code=" + i[0])

## 웹 접속, soup객체 생성

In [86]:
soups = []
for u in urls:
    res = requests.get(u)
    if res.status_code == 200:
        soup = BeautifulSoup(res.text, "lxml")
        soups.append(soup)
    else:
        print("error",u[-6:])

## 크롤링 - 현재가

In [87]:
data_code = []
data_price = []
for soup in soups:
    data_rows = soup.find("div", attrs={"id":"wrap"}).find("div", attrs={"class":"new_totalinfo"}).find_all("dd")
    c = 0
    data_category = []
    for i in data_rows:
        data_category.append(i.get_text())
    data_code.append(data_category[1][4:])
    data_price.append(data_category[3][3:11])
time = data_category[0][:-7]
time

'2021년 08월 02일 16시 11분'

In [88]:
# 주식종목과 크롤링 갯수 일치
len(data_code), len(data_price), len(code)

(40, 40, 40)

### 현재가 Data Frame 

In [89]:
t = time.split(" ")
t = "{}/{}/{}, {}:{}".format(t[0][2:4],t[1][:-1],t[2][:-1],t[3][:-1],t[4])

In [90]:
data_price_ = []
for i in data_price:
    i = i.replace("전","")
    j = i.replace(",","")
    data_price_.append(j)

In [91]:
df_summary = pd.DataFrame(data_price_, index=data_code)
df_summary.astype('float')
df_summary.columns = ["현재가 ("+t+")"]
df_summary.head()

,"현재가 (21/08/02, 16:11분)"
삼성전자,79300
LG,94800
SK하이닉스,116000
LG전자,156500
삼성전기,190500


## 크롤링 - 주식지표 

In [92]:
data = []
for soup in soups:
    data_rows = soup.find("div", attrs={"id":"aside"}).find_all("em", {"class" : None})
    for i in data_rows:
        j = i.get_text().strip()
        data.append(j)
        
#em, f_down down, f_up up

### 주식지표 데이터 정리 

In [93]:
# 전체 주식지표 - 21개씩 38개 종목 
data_value = []  
for i in data:
    j = re.sub('\n+', '', i)
    r = re.sub('\t+', '', j)
    data_value.append(r)

In [94]:
len(data_value), 21*38

(840, 798)

In [95]:
#10개만 추출
#0- 시가총액, #1- 순위, #9- 목표주가, #12 - PER, #13 - EPS, #14 - 추정PER #15 - 추정EPS, #16 - PBR, #17 - BPS, #18 - 배당수익율
nums = [0,1,9,12,13,14,15,16,17,18]
#nums2 = [38,39,47,....]

#삼성전자만 확인
for i in nums:
    print(data_value[i])

473조4,038
1
101,810
19.04
4,165
14.00
5,847
2.03
39,126
3.78


In [96]:
# 38개 항목에 대해 경제지표 21개중 10개씩 파싱
data_value_div = []
for i in range(len(data_code)):
    for j in nums:
        data_value_div.append(data_value[i*21+j])

In [97]:
len(data_value_div)

400

In [98]:
# 데이터 변환시 용이하게 ,  N/A값 변환
data_value_div_ = []
for i in data_value_div:
    j = i.replace(",","")
    k = j.replace("N/A", "0")
    data_value_div_.append(k)
data_value_div_

['473조4038',
 '1',
 '101810',
 '19.04',
 '4165',
 '14.00',
 '5847',
 '2.03',
 '39126',
 '3.78',
 '14조9121',
 '28',
 '144200',
 '8.70',
 '10896',
 '7.00',
 '13350',
 '0.86',
 '110071',
 '2.64',
 '84조4483',
 '2',
 '159286',
 '16.57',
 '7002',
 '9.00',
 '13263',
 '1.52',
 '76542',
 '1.01',
 '25조6109',
 '15',
 '219412',
 '14.58',
 '10735',
 '12.00',
 '13472',
 '1.70',
 '91952',
 '0.77',
 '14조2291',
 '30',
 '250938',
 '20.96',
 '9087',
 '15.00',
 '12757',
 '2.40',
 '79474',
 '0.74',
 '5조3133',
 '65',
 '281421',
 '13.06',
 '17195',
 '7.00',
 '30239',
 '1.97',
 '113759',
 '0.31',
 '51조3671',
 '8',
 '968333',
 '73.17',
 '10209',
 '47.00',
 '15949',
 '3.77',
 '198153',
 '0.13',
 '7조8719',
 '47',
 '32063',
 '23.31',
 '944',
 '5.00',
 '4038',
 '0.67',
 '33039',
 '0',
 '4조3762',
 '6',
 '470833',
 '33.15',
 '12517',
 '22.00',
 '19166',
 '8.34',
 '49752',
 '0.96',
 '23조6712',
 '16',
 '360100',
 '0',
 '-10575',
 '23.00',
 '11368',
 '1.49',
 '172344',
 '0',
 '60조9918',
 '5',
 '1151118',
 '37.61',
 '22

In [99]:
#각 항목별 리스트로 묶음, 2차원 리스트
data_info = []
for i in range(len(data_code)):
    data_info.append(data_value_div_[0+(i*10):10+(i*10)])
len(data_info)

40

In [100]:
for i in data_info:
    print(i)

['473조4038', '1', '101810', '19.04', '4165', '14.00', '5847', '2.03', '39126', '3.78']
['14조9121', '28', '144200', '8.70', '10896', '7.00', '13350', '0.86', '110071', '2.64']
['84조4483', '2', '159286', '16.57', '7002', '9.00', '13263', '1.52', '76542', '1.01']
['25조6109', '15', '219412', '14.58', '10735', '12.00', '13472', '1.70', '91952', '0.77']
['14조2291', '30', '250938', '20.96', '9087', '15.00', '12757', '2.40', '79474', '0.74']
['5조3133', '65', '281421', '13.06', '17195', '7.00', '30239', '1.97', '113759', '0.31']
['51조3671', '8', '968333', '73.17', '10209', '47.00', '15949', '3.77', '198153', '0.13']
['7조8719', '47', '32063', '23.31', '944', '5.00', '4038', '0.67', '33039', '0']
['4조3762', '6', '470833', '33.15', '12517', '22.00', '19166', '8.34', '49752', '0.96']
['23조6712', '16', '360100', '0', '-10575', '23.00', '11368', '1.49', '172344', '0']
['60조9918', '5', '1151118', '37.61', '22972', '16.00', '53841', '3.58', '241285', '1.16']
['6조2799', '3', '298875', '114.19', '2509', 

### 주식지표 데이터 프레임

In [101]:
data_dict = {}
for i in range(len(data_code)):
    data_dict[data_code[i]] = data_info[i] + [data_price_[i]]

df = pd.DataFrame(data_dict, 
                  index=['시가총액','시가총액순위','목표가','PER', 'EPS', '추정PER', 
                         '추정EPS', 'PBR', 'BPS', '배당수익율','현재가'+t])
df

,삼성전자,LG,SK하이닉스,LG전자,삼성전기,LG이노텍,삼성SDI,LG디스플레이,SK머티리얼즈,SK이노베이션,...,현대차,현대모비스,기아,POSCO,포스코강판,포스코인터내셔널,포스코케미칼,포스코 ICT,삼성물산,LX인터내셔널
시가총액,473조4038,14조9121,84조4483,25조6109,14조2291,5조3133,51조3671,7조8719,4조3762,23조6712,...,47조70,25조6889,34조3343,31조3873,4098,3조1522,11조9293,1조2117,26조8183,1조1415
시가총액순위,1,28,2,15,30,65,8,47,6,16,...,9,14,11,12,403,107,33,51,13,208
목표가,101810,144200,159286,219412,250938,281421,968333,32063,470833,360100,...,313864,382750,121591,471765,0,29429,190357,9800,192778,41333
PER,19.04,8.70,16.57,14.58,20.96,13.06,73.17,23.31,33.15,0,...,26.62,14.46,15.21,14.07,26.22,12.19,185.10,0,17.09,11.21
EPS,4165,10896,7002,10735,9087,17195,10209,944,12517,-10575,...,8263,18739,5567,25594,2605,2096,832,-92,8395,2626
추정PER,14.00,7.00,9.00,12.00,15.00,7.00,47.00,5.00,22.00,23.00,...,10.00,10.00,7.00,6.00,0,9.00,84.00,-111.00,15.00,6.00
추정EPS,5847,13350,13263,13472,12757,30239,15949,4038,19166,11368,...,22290,28111,11660,60777,0,2955,1829,-72,9349,5013
PBR,2.03,0.86,1.52,1.70,2.40,1.97,3.77,0.67,8.34,1.49,...,0.81,0.75,1.10,0.60,1.77,0.99,5.20,3.51,0.77,0.75
BPS,39126,110071,76542,91952,79474,113759,198153,33039,49752,172344,...,271275,363505,76884,598503,38525,25723,29588,2273,187566,39451
배당수익율,3.78,2.64,1.01,0.77,0.74,0.31,0.13,0,0.96,0,...,1.36,1.48,1.18,2.22,0.73,2.74,0.19,0.63,1.60,1.36


In [102]:
df.to_csv("output.csv")

In [103]:
DF = df.transpose()

In [104]:
print(t)
DF["현재가"+t] = DF["현재가"+t].astype('int')
DF["목표가"] = DF["목표가"].astype('int')
DF["시가총액순위"] = DF["시가총액순위"].astype('int')
DF["PER"] = DF["PER"].astype('float')
DF["EPS"] = DF["EPS"].astype('float')
DF["추정PER"] = DF["추정PER"].astype('float')
DF["추정EPS"] = DF["추정EPS"].astype('float')
DF["PBR"] = DF["PBR"].astype('float')
DF["PBR"] = DF["PBR"].astype('float')

21/08/02, 16:11분


In [105]:
DF["예상수익율"] = round((DF["목표가"]/DF["현재가"+t] - 1) *100, 2)

### 최종 DataFrame

In [106]:
DF.head()

,시가총액,시가총액순위,목표가,PER,EPS,추정PER,추정EPS,PBR,BPS,배당수익율,"현재가21/08/02, 16:11분",예상수익율
삼성전자,473조4038,1,101810,19.04,4165.0,14.0,5847.0,2.03,39126,3.78,79300,28.39
LG,14조9121,28,144200,8.70,10896.0,7.0,13350.0,0.86,110071,2.64,94800,52.11
SK하이닉스,84조4483,2,159286,16.57,7002.0,9.0,13263.0,1.52,76542,1.01,116000,37.32
LG전자,25조6109,15,219412,14.58,10735.0,12.0,13472.0,1.70,91952,0.77,156500,40.20
삼성전기,14조2291,30,250938,20.96,9087.0,15.0,12757.0,2.40,79474,0.74,190500,31.73


In [107]:
DF.index

Index(['삼성전자', 'LG', 'SK하이닉스', 'LG전자', '삼성전기', 'LG이노텍', '삼성SDI', 'LG디스플레이',
       'SK머티리얼즈', 'SK이노베이션', 'LG화학', '에코프로비엠', 'NAVER', '카카오', '이마트', '신세계',
       '롯데푸드', '신세계푸드', 'CJ제일제당', '효성', '효성티앤씨', '효성첨단소재', '효성화학', '셀트리온',
       '셀트리온헬스케어', '셀트리온제약', '셀리버리', 'SK바이오사이언스', 'SK바이오팜', '삼성바이오로직스', '현대차',
       '현대모비스', '기아', 'POSCO', '포스코강판', '포스코인터내셔널', '포스코케미칼', '포스코 ICT',
       '삼성물산', 'LX인터내셔널'],
      dtype='object')

In [108]:
DF.columns

Index(['시가총액', '시가총액순위', '목표가', 'PER', 'EPS', '추정PER', '추정EPS', 'PBR', 'BPS',
       '배당수익율', '현재가21/08/02, 16:11분', '예상수익율'],
      dtype='object')

## 반도체/전자

In [109]:
semi_elec = DF.loc[['삼성전자', 'LG', 'SK하이닉스', 'LG전자', '삼성전기', 'LG이노텍']]
semi_elec.sort_values(by="EPS", ascending=False)

,시가총액,시가총액순위,목표가,PER,EPS,추정PER,추정EPS,PBR,BPS,배당수익율,"현재가21/08/02, 16:11분",예상수익율
LG이노텍,5조3133,65,281421,13.06,17195.0,7.0,30239.0,1.97,113759,0.31,224500,25.35
LG,14조9121,28,144200,8.70,10896.0,7.0,13350.0,0.86,110071,2.64,94800,52.11
LG전자,25조6109,15,219412,14.58,10735.0,12.0,13472.0,1.70,91952,0.77,156500,40.20
삼성전기,14조2291,30,250938,20.96,9087.0,15.0,12757.0,2.40,79474,0.74,190500,31.73
SK하이닉스,84조4483,2,159286,16.57,7002.0,9.0,13263.0,1.52,76542,1.01,116000,37.32
삼성전자,473조4038,1,101810,19.04,4165.0,14.0,5847.0,2.03,39126,3.78,79300,28.39


## 반도체/소자, 디스플레이

In [110]:
semi_material = DF.loc[['삼성SDI', 'LG디스플레이', 'SK머티리얼즈']]
semi_material.sort_values(by="EPS", ascending=False)

,시가총액,시가총액순위,목표가,PER,EPS,추정PER,추정EPS,PBR,BPS,배당수익율,"현재가21/08/02, 16:11분",예상수익율
SK머티리얼즈,4조3762,6,470833,33.15,12517.0,22.0,19166.0,8.34,49752,0.96,414900,13.48
삼성SDI,51조3671,8,968333,73.17,10209.0,47.0,15949.0,3.77,198153,0.13,747000,29.63
LG디스플레이,7조8719,47,32063,23.31,944.0,5.0,4038.0,0.67,33039,0,22000,45.74


## 2차전지(전기차배터리), 화학

In [111]:
semi_material = DF.loc[['SK이노베이션', 'LG화학', '에코프로비엠','포스코케미칼']]
semi_material.sort_values(by="EPS", ascending=False)

,시가총액,시가총액순위,목표가,PER,EPS,추정PER,추정EPS,PBR,BPS,배당수익율,"현재가21/08/02, 16:11분",예상수익율
LG화학,60조9918,5,1151118,37.61,22972.0,16.0,53841.0,3.58,241285,1.16,864000,33.23
에코프로비엠,6조2799,3,298875,114.19,2509.0,79.0,3611.0,13.39,21399,0.16,286500,4.32
포스코케미칼,11조9293,33,190357,185.10,832.0,84.0,1829.0,5.20,29588,0.19,154000,23.61
SK이노베이션,23조6712,16,360100,0.00,-10575.0,23.0,11368.0,1.49,172344,0,256000,40.66


## IT/Platform

In [112]:
semi_material = DF.loc[['NAVER', '카카오']]
semi_material.sort_values(by="EPS", ascending=False)

,시가총액,시가총액순위,목표가,PER,EPS,추정PER,추정EPS,PBR,BPS,배당수익율,"현재가21/08/02, 16:11분",예상수익율
NAVER,71조2082,3,539130,4.42,98182.0,4.0,98390.0,2.91,149109,0.09,433500,24.37
카카오,64조8912,4,155364,212.52,687.0,81.0,1795.0,9.46,15428,0.02,146000,6.41


## 마트/식품

In [113]:
mart_food = DF.loc[['이마트', '신세계', '롯데푸드', '신세계푸드', 'CJ제일제당']]
mart_food.sort_values(by="EPS", ascending=False)

,시가총액,시가총액순위,목표가,PER,EPS,추정PER,추정EPS,PBR,BPS,배당수익율,"현재가21/08/02, 16:11분",예상수익율
롯데푸드,5344,342,553333,7.09,62303.0,13.0,34165.0,0.53,832397,2.71,442000,25.19
CJ제일제당,7조2034,52,616875,19.25,24858.0,12.0,39221.0,1.38,345805,0.84,478500,28.92
이마트,4조7807,68,217588,11.83,14500.0,11.0,15822.0,0.51,333019,1.17,171500,26.87
신세계,2조6287,130,393050,0.00,-3247.0,11.0,25386.0,0.74,361115,0.56,267000,47.21
신세계푸드,3795,428,133250,0.00,-3754.0,22.0,4405.0,1.33,73430,0.76,98000,35.97


## 효성

In [114]:
hyo = DF.loc[['효성', '효성티앤씨', '효성첨단소재', '효성화학']]
hyo.sort_values(by="EPS", ascending=False)

,시가총액,시가총액순위,목표가,PER,EPS,추정PER,추정EPS,PBR,BPS,배당수익율,"현재가21/08/02, 16:11분",예상수익율
효성티앤씨,3조7478,89,1138556,14.62,59220.0,5.0,159169.0,5.11,169511,0.58,866000,31.47
효성화학,1조1947,205,560000,40.89,9159.0,7.0,56568.0,2.60,144055,0,374500,49.53
효성첨단소재,2조6342,129,845400,72.57,8102.0,9.0,62831.0,7.09,82951,0,588000,43.78
효성,2조4864,135,143333,88.19,1338.0,7.0,17544.0,1.05,112207,4.24,118000,21.47


## 제약회사

In [116]:
pharm = DF.loc[['셀트리온','셀트리온헬스케어', '셀트리온제약', '셀리버리', 'SK바이오사이언스', 'SK바이오팜', '삼성바이오로직스']]
pharm.sort_values(by="EPS", ascending=False)

,시가총액,시가총액순위,목표가,PER,EPS,추정PER,추정EPS,PBR,BPS,배당수익율,"현재가21/08/02, 16:11분",예상수익율
셀트리온,36조3410,10,373333,60.45,4359.0,46.0,5670.0,10.24,25720,0,263500,41.68
삼성바이오로직스,60조4086,6,1019231,227.68,4010.0,169.0,5403.0,12.96,70428,0,913000,11.64
SK바이오사이언스,15조705,27,0,153.07,1287.0,0.0,0.0,11.68,16870,0,197000,-100.00
셀트리온헬스케어,17조4675,1,145556,89.09,1265.0,54.0,2096.0,8.76,12865,0,112700,29.15
셀트리온제약,5조3248,4,0,204.35,713.0,0.0,0.0,17.56,8298,0,145700,-100.00
SK바이오팜,9조7108,42,140000,0.00,-689.0,-357.0,-347.0,19.12,6486,0,124000,12.90
셀리버리,1조3768,42,0,0.00,-1104.0,0.0,0.0,40.07,2059,0,82500,-100.00


## 자동차

In [117]:
automotive = DF.loc[['현대차', '현대모비스', '기아']]
automotive.sort_values(by="EPS", ascending=False)

,시가총액,시가총액순위,목표가,PER,EPS,추정PER,추정EPS,PBR,BPS,배당수익율,"현재가21/08/02, 16:11분",예상수익율
현대모비스,25조6889,14,382750,14.46,18739.0,10.0,28111.0,0.75,363505,1.48,271000,41.24
현대차,47조70,9,313864,26.62,8263.0,10.0,22290.0,0.81,271275,1.36,220000,42.67
기아,34조3343,11,121591,15.21,5567.0,7.0,11660.0,1.10,76884,1.18,84700,43.55


## 포스코

In [118]:
posco = DF.loc[['POSCO', '포스코강판', '포스코인터내셔널', '포스코케미칼', '포스코 ICT',]]
posco.sort_values(by="EPS", ascending=False)

,시가총액,시가총액순위,목표가,PER,EPS,추정PER,추정EPS,PBR,BPS,배당수익율,"현재가21/08/02, 16:11분",예상수익율
POSCO,31조3873,12,471765,14.07,25594.0,6.0,60777.0,0.60,598503,2.22,360000,31.05
포스코강판,4098,403,0,26.22,2605.0,0.0,0.0,1.77,38525,0.73,68300,-100.00
포스코인터내셔널,3조1522,107,29429,12.19,2096.0,9.0,2955.0,0.99,25723,2.74,25550,15.18
포스코케미칼,11조9293,33,190357,185.10,832.0,84.0,1829.0,5.20,29588,0.19,154000,23.61
포스코 ICT,1조2117,51,9800,0.00,-92.0,-111.0,-72.0,3.51,2273,0.63,7970,22.96


## 물류

In [119]:
international = DF.loc[[ '삼성물산', 'LX인터내셔널']]
international.sort_values(by="EPS", ascending=False)

,시가총액,시가총액순위,목표가,PER,EPS,추정PER,추정EPS,PBR,BPS,배당수익율,"현재가21/08/02, 16:11분",예상수익율
삼성물산,26조8183,13,192778,17.09,8395.0,15.0,9349.0,0.77,187566,1.60,143500,34.34
LX인터내셔널,1조1415,208,41333,11.21,2626.0,6.0,5013.0,0.75,39451,1.36,29450,40.35


In [33]:
DF.to_csv("stock.csv")

In [35]:
df = pd.DataFrame({'grp': ["a", "a", "a", "a", "a", "b", "b", "b", "b", "b"], 
                   'val': np.random.uniform(0, 10, 10)})

df_sort_group_top3 = df.sort_values(by="val", ascending=False).groupby("grp").head(3)

In [36]:
df_sort_group_top3

,grp,val
0,a,9.903477
3,a,8.799277
8,b,7.846610
6,b,7.815600
9,b,4.185375
2,a,4.158718


## 시각화

### 한글폰트

In [297]:
font_name = mpl.font_manager.FontProperties(fname="/Users/jooyoungson/font_korea/malgun.ttf").get_name()
mpl.rc('font', family=font_name)
print(font_name)
plt.rc('font', family=font_name)

Malgun Gothic


In [247]:
url = "https://finance.naver.com/"
res = requests.get(url)
print(res.status_code)
res.raise_for_status()

200


In [ ]:
# f = open('stock.csv', 'w', newline='')
# write = csv.DictWriter(f, fieldnames= ["종목", "가격", "시가총액", "시가총액순위", "목표주가", 
#                                        "PER", "EPS", "추정PER", "추정EPS", "PBR","BPS" "배당수익률"])
# write.writeheader()
# write = csv.writer(f)
data_code = []
data_price = []
for soup in soups:
    data_rows = soup.find("div", attrs={"id":"wrap"}).find("div", attrs={"class":"new_totalinfo"}).find_all("dd")
    c = 0
    data_category = []
    for i in data_rows:
        data_category.append(i.get_text())
#    print(data_category[0][:21],data_category[1],data_category[3])
#    write.writerow([data_category[1][3:], data_category[3][3:10]])
    data_code.append(data_category[1][4:])
    data_price.append(data_category[3][3:11])
time = data_category[0][:-7]
#f.close()
time

In [10]:
# res = requests.get(urls[0])
# soup = BeautifulSoup(res.text, "lxml")

# data_rows = soup.find("div", attrs={"id":"wrap"}).find("div", attrs={"class":"new_totalinfo"}).find_all("dd")

# data = []
# for i in data_rows:
#     data.append(i.get_text())
# data

In [ ]:
data_index = []
for soup in soups:
    data_rows = soup.find("div", attrs={"id":"aside"}).find_all("th")
    for i in data_rows:
        data_index.append(i.get_text())
#     print(data_index[0], data_index[1], data_index[7][5:]+'\n', data_index[9][11:14]+'\n', data_index[9][15:18]+'\n', 
#           data_index[10][0:5]+'\n', data_index[10][6:9]+'\n', data_index[11][:3]+'\n', data_index[12][:5], end="\n"*2)